In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

from PIL import Image
import os

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# path
input_path = "/home/piai/ResNet/resnet_data/pilseo_new/"

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + 'validation', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

In [5]:
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()
print(labels)

tensor([180, 162, 516,  44, 169,  44, 655, 299, 501, 562,  16, 722, 501, 581,
        435, 551,  96, 279, 399, 165, 698, 100, 220, 144, 286, 510, 322, 656,
        341, 274, 216, 510])


In [6]:
classes = image_datasets['train'].classes
#classes

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
model = models.resnet101(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 831),
    #nn.LogSoftmax(dim=1) ############ 추가           
).to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [14]:
import time
import datetime

In [15]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        start = time.time()  # 시작 시간 저장
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                #return inputs, outputs, labels
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
            
        torch.save(model.state_dict(), '/media/piai/E572-EA40/psh/weights/weights_{}.h5'.format(epoch))
        sec = time.time()-start
        times = str(datetime.timedelta(seconds=sec)).split(".")
        times = times[0]
        print(times)
        print()
    return model

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=200)

Epoch 1/200
----------


/home/piai/anaconda3/envs/posco14/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train loss: 3.4952, acc: 0.3001
validation loss: 2.5434, acc: 0.4161
0:17:39

Epoch 2/200
----------
train loss: 2.4698, acc: 0.4278
validation loss: 2.2718, acc: 0.4552
0:17:17

Epoch 3/200
----------
train loss: 2.2365, acc: 0.4666
validation loss: 2.1184, acc: 0.4829
0:17:18

Epoch 4/200
----------
train loss: 2.0994, acc: 0.4885
validation loss: 1.8716, acc: 0.5298
0:17:34

Epoch 5/200
----------
train loss: 2.0064, acc: 0.5069
validation loss: 1.7581, acc: 0.5539
0:17:50

Epoch 6/200
----------
train loss: 1.9302, acc: 0.5228
validation loss: 1.7068, acc: 0.5660
0:17:34

Epoch 7/200
----------
train loss: 1.8712, acc: 0.5324
validation loss: 1.6783, acc: 0.5786
0:17:08

Epoch 8/200
----------
train loss: 1.8283, acc: 0.5422
validation loss: 1.5962, acc: 0.5891
0:17:12

Epoch 9/200
----------
train loss: 1.7916, acc: 0.5485
validation loss: 1.6509, acc: 0.5791
0:17:08

Epoch 10/200
----------
train loss: 1.7549, acc: 0.5560
validation loss: 1.5416, acc: 0.6083
0:16:47

Epoch 11/200